# Calculation of Envelope Part

In [8]:
class Side(): # マイナス側値とプラス側値をもつだけの構造体
    def __init__(self, mns, pls):
        self.mns = mns
        self.pls = pls

## Solid Layer Class

This class has the properties below:  
- thermal conductivity, W m<sup>-1</sup> K<sup>-1</sup>  
- volumetric specific heat, J m<sup>-3</sup> K<sup>-1</sup>  
- thickness, m  
- division number.

In [1]:
class Solid_Layer():
    
    def __init__(self, conductivity, specific_heat, thickness, division_number):
        if conductivity <= 0.0:
            raise ValueError('Conductivity shall be more than 0.0.')
        if specific_heat <= 0.0:
            raise ValueError('Specific Heat shall be more than 0.0.')
        if thickness <= 0.0:
            raise ValueError('Thickness shall be more than 0.0.')
        if division_number <= 0.0:
            raise ValueError('Division number shall be more than 0.')
        self.__conductivity    = conductivity
        self.__specific_heat   = specific_heat
        self.__thickness       = thickness
        self.__division_number = division_number
        self.__delta_x         = thickness / division_number
    
    @property
    def conductivity(self):
        return self.__conductivity
    
    @property
    def specific_heat(self):
        return self.__specific_heat
    
    @property
    def thickness(self):
        return self.__thickness
    
    @property
    def division_number( self ):
        return self.__division_number
    
    @property
    def delta_x(self):
        return self.__delta_x

In [4]:
s = Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=10)

In [6]:
s.conductivity, s.specific_heat, s.thickness, s.division_number, s.delta_x

(1.2, 100.0, 0.12, 10, 0.012)

In [47]:
class Air_Layer():
    
    def __init__(self, resistance):
        if resistance <= 0.0:
            raise ValueError('Air resistance shall be more than 0.0.')
        self.__resistance = resistance
        
    @property
    def resistance(self):
        return self.__resistance

In [48]:
a = Air_Layer(resistance=0.1)
a.resistance

0.1

## Cell Class

In [2]:
class Cell():
    
    def __init__(self, resistance, capacity):
        self.__resistance = resistance
        self.__capacity   = capacity
    
    @property
    def resistance(self):
        return self.__resistance
    
    @property
    def capacity(self):
        return self.__capacity
    
    def set_initial_temperature(self, initial_temperature = 0.0):
        self.__temperature = initial_temperature
        
    @property
    def temperature(self):
        return self.__temperature
    
    def add_adjacent_cells(self, adjacent_cells ):
        self.adjacent_cells = adjacent_cells
        
    def calc_new_temperature(self, side_temperature, surf_resistance, delta_t):
        
        temp_adj = Side(mns = side_temperature.mns if self.adjacent_cells.mns == 'AirMns' else adjacent_cells.mns.temperature,
                        pls = side_temperature.pls if self.adjacent_cells.pls == 'AirPls' else adjacent_cells.pls.temperature)
        
        resistance_adj = Side(mns = surf_resistance.mns if self.adjacent_cells.mns == 'AirMns' else self.resistance.mns,
                              pls = surf_resistance.pls if self.adjacent_cells.pls == 'AirPls' else self.resistance.pls)
        
        heatflow = ( ( temp_adj.mns - self.tempetature ) / resistance_adj.mns + ( temp_adj.pls - self.tempetature ) / resistance_adj.pls ) * delta_t
        
        self.__newtemp = heatflow / self.capacity + self.temperature
        
    def updatetemp( self ):
        
        self.__temperature = self.__newtemperature


## Envelope Class

In [115]:
s = Solid_Layer(1.2,1.2,1.2,1)
type(s) == Solid_Layer

True

In [144]:
class Envelope():
    
    def __init__(self):
        
        self.__is_parameters_set  = False
        self.__is_temperature_set = False
    
    def set_parameters(self, nameset, layers):
        
        self.__nameset = nameset
        
        if type(layers[0]) == Air_Layer:
            raise Exception('First layer should not be air layer.')
        
        if type(layers[len(layers)-1]) == Air_Layer:
            raise Exception('Last layer should not be air layer.')
        
        for i, l in enumerate(layers):
            if type(l) == Air_Layer:
                if type(layers[i+1]) == Air_Layer:
                    raise Exception('Air layer should not set next to the air layer.')
        
        self.__cells = []
        
        for i, layer in enumerate(layers):
            if type(layer) == Solid_Layer:
                
                if i == 0:
                    self.__cells.append(Cell(resistance = Side(mns = 'surfaceMns', pls = layer.delta_x / layer.conductivity),
                                             capacity   = layer.specific_heat * layer.delta_x / 2
                                            ))
                elif type(layers[i-1]) == Air_Layer:
                    self.__cells.append(Cell(resistance = Side(mns = layers[i-1].resistance, pls = layer.delta_x / layer.conductivity),
                                             capacity   = layer.specific_heat * layer.delta_x / 2
                                            ))
                else:
                    pass
                
                self.__cells.extend([Cell(resistance = Side(mns = layer.delta_x / layer.conductivity, pls = layer.delta_x / layer.conductivity),
                                          capacity   = layer.specific_heat * layer.delta_x
                                         )] * (l.division_number - 1))
                
                if i == len(layers) - 1:
                    self.__cells.append(Cell(resistance = Side(mns = layer.delta_x / layer.conductivity, pls = 'surfacePls'),
                                             capacity   = layer.specific_heat * layer.delta_x / 2
                                            ))
                elif type(layers[i+1]) == Air_Layer:
                    self.__cells.append(Cell(resistance = Side(mns = layer.delta_x / layer.conductivity, pls = layers[i+1].resistance),
                                             capacity   = layer.specific_heat * layer.delta_x / 2
                                            ))
                else:
                    self.__cells.append(Cell(resistance = Side(mns = layer.delta_x / layer.conductivity, pls = layers[i+1].delta_x / layers[i+1].conductivity),
                                             capacity   = layer.specific_heat * layer.delta_x / 2 + layers[i+1].specific_heat * layers[i+1].delta_x / 2
                                            ))
        
        for i, cell in enumerate(self.__cells):
            if i == 0:
                cell.add_adjacent_cells(Side(mns = 'surfaceMns', pls = self.__cells[i+1]))
            elif i == len(self.__cells)-1:
                cell.add_adjacent_cells(Side(mns = self.__cells[i-1], pls = 'surfacePls'))
            else:
                cell.add_adjacent_cells(Side(mns = self.__cells[i-1], pls = self.__cells[i+1]))
                        
        self.__is_parameters_set = True
                
    def set_initial_temperature(self, set_temperature):
        if not self.__is_parameters_set:
            raise Exception( 'error!: Before setting the initial temperature, call SetParameters function.' )
        [ cell.set_initial_temperature(set_temperature) for c in self.__cells ]
        self.__is_temperature_set = True
        
    def show_temperature(self, index):
        self.check_set_initial_temperature():
        if index >= len(self.__cells):
            raise Exception('Index is over the number of cells.')
        if index < 0:
            raise Exception('Index should be over zero.')
        return self.__cells[index].temperature
        
    def check_set_initial_temperature(self):
        if not self.__is_temperature_set:
            raise Exception('Set initial temperature!')

    def show_surface_temperature(self):
        self.check_set_initial_temperature()
        return (self.__cells[0].temperature, self.__cells[-1].temperature)

    def calc(self, rtemp, htransferc, sunabs, swave, lwaveemit, lwave, dt):
        surface_heatflow_mns = htransferc.mns * (rtemp.mns - self.__cells[0].temperature ) \
                               + lwave.mns * lwaveemit.mns - self.HeatRadiation(lwaveemit.mns) \
                               + swave.mns * sunabs.mns

    def calc_convective_heat_transfer(self, air_temp, heat_transfer_coeff):
        
        heat_mns = heat_transfer_coeff.mns * (air_temp.mns - self.__cells[0])
        heat_pls = heat_transfer_coeff.pls * (air_temp.pls - self.__cells[-1])
        
        return Side(heat_mns, heat_pls)
    
    def calc_heat_radiation
        /// <summary>
        /// 壁体表面から両側に放射される熱流を示す
        /// </summary>
        /// <remarks>温度が初期化されていない場合エラーとなる</remarks>
        /// <param name="lwaveemit">壁体両側の長波長放射率</param>
        /// <returns>壁体から放射される長波長放射 [W/m2]</returns>
        public BothSide<double> HeatRadiation(BothSide<double> lwaveemit)
        {
            CheckSetInitialTempSet();

            BothSide<double> lwave;

            lwave.sideA = CalcLWaveEmitFromTemp(lwaveemit.sideA, SurfaceTempShow().sideA);
            lwave.sideB = CalcLWaveEmitFromTemp(lwaveemit.sideB, SurfaceTempShow().sideB);

            return lwave;

        }

    def check_longwave_emittance_range(self, value):
        if value < 0.0 or value > 1.0:
            raise ValueError('Long wave emittance should be in the range between 0.0 and 1.0.')

    def calc_longwave_emittance(self, lwaveemit, surftemp):
        self.check_longwave_emittance_range(lwaveemit)
        SBFUNC = 5.670367 * 10**(-8)
        TPWATER = 273.15
        return lwaveemit * SBFUNC * (surftemp + TPWATER)**4


In [145]:
e = Envelope()
e.set_parameters('test',[Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=3),
                         Air_Layer(resistance=120.0),
                         Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=3),
                         Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=3),
                         Air_Layer(resistance=120.0),
                         Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=6),
                        ])
e._Envelope__is_parameters_set, e._Envelope__is_temperature_set,

(True, False)

In [146]:
for i in range(3):
    print(i)

0
1
2


In [150]:
for c in e.cells:
    print(c.resistance)
    print(c.capacity)


2.0
4.0
4.0
4.0
4.0
4.0
2.0
2.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
2.0
1.0
2.0
2.0
2.0
2.0
2.0
1.0


In [113]:
e._Envelope__layers

AttributeError: 'Envelope' object has no attribute '_Envelope__layers'

In [ ]:

    def Calc(rtemp, htransferc, sunabs, swave, lwaveemit, lwave, dt):
        
        self.CheckSetInitialTempSet()
        
        n, m, k = 0, 0, 0

        # 壁体表面の計算
        
        self.__newtemperatures[n] = self.__temperatures[n] + 2 * dt / self.__solidLayers[m].getSpecificHeat() / self.__solidLayers[m].getDeltaX() \
                                    + (htranserc[0] * (rtemp[0] - self.temperatures[n])
                                       + lwave[0] * lwaveemit[0] - self.HeatRadiation(lwaveemit)[0]
                                       + swave[0] * sunabs[0]
                                       + self.__solidLayers[m].getConductivity() * (self.__temperatures[n+1] - self.__temperatures[n]) / self.__solidLayers[m].getDeltaX()
                                      )
        
        n = n + 1
        
        
        

        public void Calc(BothSide<double> rtemp, BothSide<double> htransferc, BothSide<double> sunabs, BothSide<double> swave,
                         BothSide<double> lwaveemit, BothSide<double> lwave, double dt)
        {

            for (m = 0; m < _layers.Length; m++)
            {
                // 分割数が2以上の時
                if (_layers[m].Div != 1)
                {

                    // 計算＠両側の質点が同じ材質（物性値）

                    for (k = 0; k < _layers[m].Div - 1; k++)
                    {
                        newtemp[n] = temp[n] + dt / _layers[m].Sph / _layers[m].Deltax
                            * _layers[m].Cond * (temp[n + 1] + temp[n - 1] - 2 * temp[n]) / _layers[m].Deltax;
                        n = n + 1;
                    }
                }

                if (_apos == m + 1)

                // 計算＠層と空気層の間
                {
                    newtemp[n] = temp[n] + 2 * dt / _layers[m].Sph / _layers[m].Deltax
                        * (_layers[m].Cond * (temp[n - 1] - temp[n]) / _layers[m].Deltax
                        + (temp[n + 1] - temp[n]) / _aregi);
                    n = n + 1;
                    newtemp[n] = temp[n] + 2 * dt / _layers[m + 1].Sph / _layers[m+1].Deltax
                        * ((temp[n - 1] - temp[n]) / _aregi
                        + _layers[m + 1].Cond * (temp[n + 1] - temp[n]) / _layers[m+1].Deltax);
                    n = n + 1;
                }

                else if (m != _layers.Length - 1)


                // 計算＠異種材料間（空気層なし）
                {
                    newtemp[n] = temp[n]
                        + 2 * dt / (_layers[m].Sph * _layers[m].Deltax + _layers[m + 1].Sph * _layers[m+1].Deltax)
                        * (_layers[m].Cond * (temp[n - 1] - temp[n]) / _layers[m].Deltax
                        + _layers[m + 1].Cond * (temp[n + 1] - temp[n]) / _layers[m+1].Deltax);

                    n = n + 1;
                }

            }

            // 計算＠壁体表面
            newtemp[n] = temp[n] + 2 * dt / _layers[m - 1].Sph / _layers[_layers.Length - 1].Deltax
                * (htransferc.sideB * (rtemp.sideB - temp[n])          // 対流
                + lwave.sideB * lwaveemit.sideB - HeatRadiation(lwaveemit).sideB // 長波長放射
                + swave.sideB * sunabs.sideB                          // 日射
                + _layers[m - 1].Cond * (temp[n - 1] - temp[n]) / _layers[_layers.Length - 1].Deltax); // 伝導成分

            // 新しい壁体温度に置き換える
            // temp = newtemp;

        }

        public void Update()
        {
            temp = newtemp;
        }






           

In [ ]:
w = Wall()

In [ ]:
w._Wall__isParametersSet

In [ ]:
w._Wall__isTemperatureSet

In [ ]:
w.SetParameters('',[ SolidLayer( 1.0, 1.0, 1.0, 5 ), SolidLayer( 1.0, 1.0, 1.0, 5 ), SolidLayer( 1.0, 1.0, 1.0, 5 ), SolidLayer( 1.0, 1.0, 1.0, 5 ) ],3,0.08)
w._Wall__mesh
w.SetInitialTemperature( 1.5 )
w._Wall__temperatures

In [ ]:
w.SetParameters( '',[ SolidLayer( 1.0, 1.0, 1.0, 5 ), SolidLayer( 1.0, 1.0, 1.0, 5 )  ], 0, 0.08)
w._Wall__mesh
w._Wall__surfaceNumber

In [ ]:
namespace HeatLoadCalc.Logic
{
    public class Wall
    {


    }
}
